In [132]:
from pymavlink import mavutil
import time
from datetime import datetime, timezone
from influxdb import InfluxDBClient, DataFrameClient
import pandas as pd

In [25]:
path = r"C:\Users\Alton\Downloads\drive-download-20201106T043512Z-001\LOGGER03.CSV"

df = pd.read_csv(path)

In [163]:
# look for gps fix
idx_gps = (df["gpsfix"] == 1)

df_gpstime = df.loc[idx_gps][["year", "month", "day", "hour", "min", "second"]]
df_gpstime = df_gpstime.rename(columns={'min': 'minute'})

df_gps = df.loc[idx_gps][["boardtime", "gpsfix", "numsats", "hdop", "age", "lat", "lon"]]
df_gps["datetime"] = pd.to_datetime(df_gpstime)

datum_gps = df_gps["datetime"].iloc[0]
datum_board = df_gps["boardtime"].iloc[0]

# simple method
delta_gps = (df_gps["datetime"].iloc[-1]-datum_gps).total_seconds()
delta_board = (df_gps["boardtime"].iloc[-1]-datum_board)/1000

factor = delta_gps/delta_board

df["newboardtime"] = ((df["boardtime"]-datum_board)*factor)
df["datetime"] = datum_gps + pd.to_timedelta(df["newboardtime"], unit="ms")

df_gnd_weather = df.loc[~idx_gps][["datetime", "winddir", "windspeed", "humidity", "tempf", "pressure", "batt_lvl", "light"]]
df_gnd_weather["tempc"] = (df_gnd_weather["tempf"] - 32) * 5 / 9



In [131]:
client = InfluxDBClient(host='192.168.0.99', port=8086)
###client.drop_database('pyexample')
client.create_database('pyexample')
print(client.get_list_database())
client.switch_database('pyexample')

[{'name': '_internal'}, {'name': 'pyexample'}]


In [202]:
for idx, row in df_gnd_weather.iterrows():
    dict_body = {}
    dict_body["measurement"] = "Ground_Weather"
    dict_body["time"] = row["datetime"].isoformat()
    row.pop("datetime")
    row.pop("tempf")
    dict_body["fields"] = row.to_dict()
    json_body = [dict_body]

    
    
    # WRITE TO DB
    client.write_points(json_body)
    print(dict_body["time"])


T20:48:01.946226740
2020-11-04T20:48:02.953453206
2020-11-04T20:48:03.962680121
2020-11-04T20:48:04.969906587
2020-11-04T20:48:05.976132828
2020-11-04T20:48:06.987360193
2020-11-04T20:48:07.999587783
2020-11-04T20:48:09.006814248
2020-11-04T20:48:10.835225350
2020-11-04T20:48:11.889462385
2020-11-04T20:48:12.897689076
2020-11-04T20:48:13.905915766
2020-11-04T20:48:14.918143356
2020-11-04T20:48:15.925369822
2020-11-04T20:48:16.946599436
2020-11-04T20:48:17.954826126
2020-11-04T20:48:18.962052592
2020-11-04T20:48:19.971279507
2020-11-04T20:48:20.980506422
2020-11-04T20:48:22.013738736
2020-11-04T20:48:23.022965651
2020-11-04T20:48:24.031192341
2020-11-04T20:48:25.859603443
2020-11-04T20:48:26.936845651
2020-11-04T20:48:27.945072341
2020-11-04T20:48:28.959300381
2020-11-04T20:48:29.963526172
2020-11-04T20:48:30.971752863
2020-11-04T20:48:31.995983151
2020-11-04T20:48:33.003209617
2020-11-04T20:48:34.012436532
2020-11-04T20:48:35.020663223
2020-11-04T20:48:36.032890813
2020-11-04T20:48:37.